# Homework Week 1
## Q1. Running Elastic
Run Elastic Search 8.4.3, and get the cluster information.

In [1]:
! curl localhost:9200

{
  "name" : "42f0e0226508",
  "cluster_name" : "docker-cluster",
  "cluster_uuid" : "vmz5hGo1RrShON2Nb0_DYA",
  "version" : {
    "number" : "8.4.3",
    "build_flavor" : "default",
    "build_type" : "docker",
    "build_hash" : "42f05b9372a9a4a470db3b52817899b99a76ee73",
    "build_date" : "2022-10-04T07:17:24.662462378Z",
    "build_snapshot" : false,
    "lucene_version" : "9.3.0",
    "minimum_wire_compatibility_version" : "7.17.0",
    "minimum_index_compatibility_version" : "7.0.0"
  },
  "tagline" : "You Know, for Search"
}


In [2]:
# alternatively we can use following to get info about running elastic search engine
from elasticsearch import Elasticsearch

es_client = Elasticsearch("http://localhost:9200")
es_client.info()

ObjectApiResponse({'name': '42f0e0226508', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'vmz5hGo1RrShON2Nb0_DYA', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

Q: What's the version.build_hash value?

A: It's a hash of build version of running elastic search engine.

In [3]:
import requests

docs_url = "https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1"
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course["course"]

    for doc in course["documents"]:
        doc["course"] = course_name
        documents.append(doc)

In [4]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

## Q2. Indexing the data

In [5]:
! pip install elasticsearch

Q: Which function do you use for adding your data to elastic?

A: Using `index` method we can add data to elastic search search engine client

In [6]:
index_settings = {
    "settings": {"number_of_shards": 1, "number_of_replicas": 0},
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
        }
    },
}

Create a index name `course-questions-homework` and add data to it

In [7]:
index_name = "course-questions-homework"
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions-homework'})

In [8]:
from tqdm import tqdm

for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|██████████| 948/948 [00:03<00:00, 268.51it/s]


## Q3. Searching

In [9]:
query = "How do I execute a command in a running docker container?"

search_query = {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields",
                }
            },
        }
    },
}

In [10]:
response = es_client.search(index=index_name, body=search_query)

In [11]:
response["hits"]["hits"][0]

{'_index': 'course-questions-homework',
 '_id': 'Hf4acpAB3eoYInjJ06LE',
 '_score': 81.79147,
 '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I debug a docker container?',
  'course': 'machine-learning-zoomcamp'}}

Q: What's the score for the top ranking result?

A:  83.135956

## Q4. Filtering

Filter the questions to `machine-learning-zoomcamp`

In [12]:
search_query = {
    "size": 3,  # return top 3 results
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields",
                }
            },
            "filter": {"term": {"course": "machine-learning-zoomcamp"}},
        }
    },
}

In [13]:
response = es_client.search(index=index_name, body=search_query)

In [14]:
response["hits"]["hits"]

[{'_index': 'course-questions-homework',
  '_id': 'Hf4acpAB3eoYInjJ06LE',
  '_score': 81.79147,
  '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
   'section': '5. Deploying Machine Learning Models',
   'question': 'How do I debug a docker container?',
   'course': 'machine-learning-zoomcamp'}},
 {'_index': 'course-questions-homework',
  '_id': 'PP4acpAB3eoYInjJ1KIw',
  '_score': 50.006645,
  '_source': {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /p

In [15]:
# 3rd result
response["hits"]["hits"][-1]

{'_index': 'course-questions-homework',
 '_id': 'Pf4acpAB3eoYInjJ1KI0',
 '_score': 48.541283,
 '_source': {'text': 'You can copy files from your local machine into a Docker container using the docker cp command. Here\'s how to do it:\nIn the Dockerfile, you can provide the folder containing the files that you want to copy over. The basic syntax is as follows:\nCOPY ["src/predict.py", "models/xgb_model.bin", "./"]\t\t\t\t\t\t\t\t\t\t\tGopakumar Gopinathan',
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I copy files from a different folder into docker container’s working directory?',
  'course': 'machine-learning-zoomcamp'}}

In [16]:
response["hits"]["hits"][-1]["_source"]["question"]

'How do I copy files from a different folder into docker container’s working directory?'

## Q5. Building a Prompt

In [17]:
def build_context(response):
    context_template = """
    Q: {question}
    A: {text}
    """.strip()
    context = ""
    for entry in response["hits"]["hits"]:
        text = entry["_source"]["text"]
        question = entry["_source"]["question"]

        context += context_template.format(question=question, text=text) + "\n\n"
    return context

In [18]:
context = build_context(response)

In [19]:
context

'Q: How do I debug a docker container?\n    A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)\n\nQ: How do I copy files from my local machine to docker container?\n    A: You can copy files from your local machine into a Docker container using the docker cp command. Here\'s how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\nHrithik Kumar Advani\n\nQ: How do I copy files from a different folder into docker container’s working directory?\n    A: You can copy files from your local machine into a Docker container using the dock

In [20]:
def build_prompt(question, context):
    prompt_template = """
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
    Use only the facts from the CONTEXT when answering the QUESTION.

    QUESTION: {question}

    CONTEXT:
    {context}
    """.strip()

    return prompt_template.format(question=question, context=context)

In [21]:
query = "How do I execute a command in a running docker container?"

prompt = build_prompt(question=query, context=context)

In [22]:
len(prompt)

1492

## Q6. Tokens

When we use the OpenAI Platform, we're charged by the number of tokens we send in our prompt and receive in the response.

The OpenAI python package uses `tiktoken` for tokenization:

In [23]:
! pip install tiktoken

Let's calculate the number of tokens in our query:

In [24]:
import tiktoken

encoding = tiktoken.encoding_for_model("gpt-4o")

Number of tokens for `prompt` variable

In [25]:
len(encoding.encode(prompt))

330

In [26]:
encoding.decode(encoding.encode(prompt))

'You\'re a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.\n    Use only the facts from the CONTEXT when answering the QUESTION.\n\n    QUESTION: How do I execute a command in a running docker container?\n\n    CONTEXT:\n    Q: How do I debug a docker container?\n    A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)\n\nQ: How do I copy files from my local machine to docker container?\n    A: You can copy files from your local machine into a Docker container using the docker cp command. Here\'s how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /p

## Bonus: generating the answer using OpenAI API

In [27]:
import os
from openai import OpenAI
from dotenv import load_dotenv

# Load .env file
load_dotenv()
API_KEY = os.getenv("API_KEY")

client = OpenAI(api_key=API_KEY)


def gpt(prompt):
    response = client.chat.completions.create(
        model="gpt-4o", messages=[{"role": "user", "content": prompt}]
    )

    return response.choices[0].message.content

In [28]:
gpt(prompt)

'To execute a command in a running Docker container, follow these steps:\n\n1. Use `docker ps` to find the `container-id` of the running container.\n2. Execute your command in the specific container using `docker exec`.\n\nFor example, to start a bash session inside the running container, you can use:\n\n```bash\ndocker exec -it <container-id> bash\n```'